<a href="https://colab.research.google.com/github/nrm176/japan-trade-stats/blob/master/%E6%99%AE%E9%80%9A%E8%B2%BF%E6%98%93%E7%B5%B1%E8%A8%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get started

### Step1.
Access to [e-stat](https://www.e-stat.go.jp/stat-search/files?page=1&layout=datalist&toukei=00350300&tstat=000001013143&cycle=1&year=20200&month=12040605&tclass1=000001013229&tclass2=000001013230&result_back=1&cycle_facet=cycle)

### Step2.
Using Chrome DevTool, run the following JavaScript snippet.  It will return the urls to download csv files which include custom data.

```javascript
Array.from(document.querySelector('div.stat-dataset_list')
    .querySelectorAll('li.stat-dataset_list-detail-item > div div:nth-child(4) > a:nth-child(1)'))
    .map((e) => {
    return e.href
})
```

### Step3.

Copy and paste those urls you got from Step2 and assign them to the variable with name urls

In [ ]:
import pandas as pd
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
urls = [
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957311&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957312&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957313&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957314&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957315&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957316&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957317&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957318&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957319&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957320&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957321&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957322&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957323&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957324&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957325&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957326&fileKind=1",
  "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031957327&fileKind=1"
]

In [ ]:
dfs = []
for url in urls:
  dfs.append(pd.read_csv(url))

In [ ]:
columns = list(dfs[0].columns)

In [ ]:
quantity_columns = list(filter(lambda x: 'Quantity' in x, columns))
value_columns = list(filter(lambda x: 'Value' in x, columns))

In [ ]:
common_columns = ['Year', 'HS', 'Country', 'Custom']

In [ ]:
value_dfs = []
for df in dfs:
  value_dfs.append(df[ common_columns + value_columns ])

In [ ]:
value_df_combined = pd.concat(value_dfs)

In [ ]:
value_df_combined['HS'] = list(map(lambda x: x.replace("'", ""), list(value_df_combined['HS'])))

In [ ]:
value_df_combined['Custom'] = value_df_combined['Custom'].astype(str)

In [ ]:
county_code_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/trade-stat/datasets_262928_1193059_country_eng.csv', index_col=0)

In [ ]:
hs9_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/trade-stat/datasets_262928_1193059_hs9_jpn.csv', index_col=0, converters={'hs9': lambda x: str(x)})
hs9_df = hs9_df.rename(columns={"hs9": "HS", "hs9_name": "HS_name"})

In [ ]:
custom_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/trade-stat/custom-names.csv', converters={'custom_code': lambda x: str(x)})
custom_df = custom_df.rename(columns={"custom_code": "Custom", "custom_name": "CustomName"})

In [ ]:
df_with_hs9 = pd.merge(value_df_combined, hs9_df, how='inner', on='HS' )

In [ ]:
df_with_hs9_custom = pd.merge(df_with_hs9, custom_df, how='inner', on='Custom')
df_with_hs9_custom_country = pd.merge(df_with_hs9_custom, county_code_df, how='inner', on='Country' )

In [ ]:
april_value_not_zero = df_with_hs9_custom_country['Value-Apr'] != 0
df_with_hs9_custom_country_apr_not_zero = df_with_hs9_custom_country[april_value_not_zero]

In [ ]:
ten_percent_increase_from_apr = (df_with_hs9_custom_country_apr_not_zero['Value-May']-df_with_hs9_custom_country_apr_not_zero['Value-Apr'])/df_with_hs9_custom_country_apr_not_zero['Value-Apr'] * 100 > 10

In [ ]:
df2 = df_with_hs9_custom_country_apr_not_zero[ten_percent_increase_from_apr][['Value-Feb','Value-Mar','Value-Apr', 'Value-May', 'HS_name', 'CustomName', 'Country_name']]

In [ ]:
df2[ (df2['Value-May'] > 1000000) & (df2['HS_name'].str.contains('半導体')) ].sort_values(by='Value-May', ascending=False)

,Value-Feb,Value-Mar,Value-Apr,Value-May,HS_name,CustomName,Country_name
17589,23679478,17848987,18726452,27752424,半導体デバイス又は集積回路製造用の機器,下関税関支署,Korea
49166,1383150,787985,597090,6431639,半導体デバイス又は集積回路製造用の機器,東京税関（本関）,Taiwan
36471,6644245,4220696,1268263,4657310,半導体デバイス又は集積回路製造用の機器,関西空港税関支署,China
17714,435769,1506198,1266586,2707818,半導体ウエハー又は半導体デバイスの検査用の機器及びフォトマスク又はレチクル（半導体デバイスの...,下関税関支署,Korea
55549,1732178,1518399,1764345,2210714,半導体デバイス又は集積回路製造用の機器,神戸税関（本関）,Taiwan
46530,2581801,450908,1153230,2142500,半導体デバイス又は集積回路製造用の機器,下関税関支署,China
200022,0,0,1337059,1631314,半導体デバイス又は集積回路製造用の機器,東京税関（本関）,Ireland
36470,19170,656200,249380,1474700,半導体ボール又は半導体ウエハー製造用の機器,関西空港税関支署,China
115999,807481,957191,640412,1370697,その他のもの ディスク、テープ、不揮発性半導体記憶装置、スマートカードその他の媒体（記録して...,東京税関成田航空貨物出張所,Singapore
183713,2698720,162659,3930,1343584,半導体デバイス又は集積回路製造用の機器,東京税関成田航空貨物出張所,Israel


In [ ]:
df_with_hs9_custom_country_apr_not_zero[ (df_with_hs9_custom_country_apr_not_zero['HS_name'].str.contains('半導体')) ][['Value-Mar', 'Value-Apr','Value-May', 'CustomName']].groupby('CustomName').sum().sort_values(by='Value-May', ascending=False)

,Value-Mar,Value-Apr,Value-May
CustomName,,,
東京税関成田航空貨物出張所,68416308,47204635,42403387
下関税関支署,21991012,22357142,34280971
東京税関（本関）,9549734,11948865,16613916
関西空港税関支署,14979999,8212606,10885416
神戸税関（本関）,6322703,6473002,5060370
大阪税関（本関）,5785973,4599164,3053859
横浜税関（本関）,4048380,4382392,2962044
名古屋税関（本関）,4331540,5472545,2286247
呉税関支署,1929917,2641235,2145612
